# Ejercicio 2 - Arquitectura de Red Multiperceptrón para Clasificación de Semillas

Se desea utilizar una red multiperceptrón para reconocer muestras de tres variedades diferentes de trigo:
**Kama, Rosa y Canadiense**. Para entrenarla se utilizará una parte de los ejemplos del archivo `SEMILLAS.CSV`.

Fuente de datos: Seeds Data Set - https://archive.ics.uci.edu/ml/datasets/seeds

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Cargar y explorar los datos

In [ ]:
# Cargar el dataset
df = pd.read_csv('../../Datos/semillas.csv')

print("Primeras filas del dataset:")
print(df.head())
print(f"\nDimensiones del dataset: {df.shape}")
print(f"\nColumnas: {list(df.columns)}")

In [ ]:
# Información del dataset
print("Información del dataset:")
print(df.info())
print("\nClases disponibles:")
print(df['Clase'].value_counts())

## a) Análisis de la Arquitectura de la Red

Para diseñar la arquitectura de una red neuronal multiperceptrón, necesitamos determinar:
1. Cantidad de neuronas en la capa de entrada
2. Cantidad de neuronas en la capa de salida
3. Cantidad total de pesos (arcos) en la red

In [ ]:
# Separar características (X) y etiquetas (y)
X = df.drop('Clase', axis=1)
y = df['Clase']

# Número de características (atributos de entrada)
num_features = X.shape[1]
print(f"Características de entrada: {list(X.columns)}")
print(f"\nNúmero total de características: {num_features}")

# Número de clases
num_classes = y.nunique()
print(f"\nClases a reconocer: {sorted(y.unique())}")
print(f"Número total de clases: {num_classes}")

## Respuestas:

### 1. Cantidad de neuronas de la capa de entrada

La capa de entrada debe tener **una neurona por cada característica** del dataset.

Las características son:
- Area
- Perimetro
- Compacidad
- LongNucleo
- AnchoNucleo
- Asimetria
- LongSurco

**Respuesta: 7 neuronas en la capa de entrada**

In [ ]:
# Capa de entrada
neuronas_entrada = num_features
print(f"Neuronas en la capa de entrada: {neuronas_entrada}")

### 2. Cantidad de neuronas de la capa de salida

Para un problema de **clasificación multiclase**, la capa de salida debe tener **una neurona por cada clase** a reconocer.

Las clases son:
- Kama (Tipo1)
- Rosa (Tipo2)
- Canadiense (Tipo3)

**Respuesta: 3 neuronas en la capa de salida**

In [ ]:
# Capa de salida
neuronas_salida = num_classes
print(f"Neuronas en la capa de salida: {neuronas_salida}")

### 3. Cantidad de pesos (arcos) con una capa oculta de 4 neuronas

La arquitectura de la red es:
- **Capa de entrada**: 7 neuronas
- **Capa oculta**: 4 neuronas
- **Capa de salida**: 3 neuronas

Los pesos se calculan como:

1. **Pesos entre capa de entrada y capa oculta**:
   - Cada neurona de entrada se conecta con cada neurona de la capa oculta
   - Pesos = 7 × 4 = 28
   - Bias de la capa oculta = 4 (uno por neurona)
   - **Total capa 1**: 28 + 4 = **32 pesos**

2. **Pesos entre capa oculta y capa de salida**:
   - Cada neurona de la capa oculta se conecta con cada neurona de salida
   - Pesos = 4 × 3 = 12
   - Bias de la capa de salida = 3 (uno por neurona)
   - **Total capa 2**: 12 + 3 = **15 pesos**

**Respuesta: Total de pesos = 32 + 15 = 47 pesos**

In [ ]:
# Capa oculta
neuronas_oculta = 4

# Calcular pesos
# Pesos entre entrada y capa oculta (incluyendo bias)
pesos_entrada_oculta = (neuronas_entrada * neuronas_oculta) + neuronas_oculta

# Pesos entre capa oculta y salida (incluyendo bias)
pesos_oculta_salida = (neuronas_oculta * neuronas_salida) + neuronas_salida

# Total de pesos
total_pesos = pesos_entrada_oculta + pesos_oculta_salida

print(f"Arquitectura de la red:")
print(f"  Capa de entrada: {neuronas_entrada} neuronas")
print(f"  Capa oculta: {neuronas_oculta} neuronas")
print(f"  Capa de salida: {neuronas_salida} neuronas")
print(f"\nCálculo de pesos:")
print(f"  Entre entrada y capa oculta: {neuronas_entrada} × {neuronas_oculta} + {neuronas_oculta} (bias) = {pesos_entrada_oculta}")
print(f"  Entre capa oculta y salida: {neuronas_oculta} × {neuronas_salida} + {neuronas_salida} (bias) = {pesos_oculta_salida}")
print(f"\n{'='*60}")
print(f"TOTAL DE PESOS (ARCOS) EN LA RED: {total_pesos}")
print(f"{'='*60}")

## Visualización de la arquitectura

In [ ]:
# Crear un diagrama visual de la arquitectura
fig, ax = plt.subplots(figsize=(12, 8))

# Posiciones de las capas
x_positions = [0, 2, 4]
layer_sizes = [neuronas_entrada, neuronas_oculta, neuronas_salida]
layer_names = ['Entrada\n(7 neuronas)', 'Oculta\n(4 neuronas)', 'Salida\n(3 neuronas)']

# Dibujar neuronas
for layer_idx, (x, size, name) in enumerate(zip(x_positions, layer_sizes, layer_names)):
    y_positions = np.linspace(0, 6, size)
    for y in y_positions:
        circle = plt.Circle((x, y), 0.2, color='lightblue', ec='darkblue', linewidth=2)
        ax.add_patch(circle)
    
    # Agregar nombre de la capa
    ax.text(x, -1, name, ha='center', fontsize=12, fontweight='bold')

# Dibujar conexiones (solo algunas para no saturar el gráfico)
for i, x1 in enumerate(x_positions[:-1]):
    size1 = layer_sizes[i]
    size2 = layer_sizes[i+1]
    y1_positions = np.linspace(0, 6, size1)
    y2_positions = np.linspace(0, 6, size2)
    
    for y1 in y1_positions:
        for y2 in y2_positions:
            ax.plot([x_positions[i]+0.2, x_positions[i+1]-0.2], 
                   [y1, y2], 'gray', alpha=0.3, linewidth=0.5)

# Agregar información de pesos
ax.text(1, 7, f'Pesos: {pesos_entrada_oculta}\n(28 + 4 bias)', 
        ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax.text(3, 7, f'Pesos: {pesos_oculta_salida}\n(12 + 3 bias)', 
        ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

ax.set_xlim(-0.5, 4.5)
ax.set_ylim(-2, 8)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title(f'Arquitectura de la Red Neuronal\nTotal de pesos: {total_pesos}', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

## Resumen de Respuestas

### a) Con respecto a la arquitectura:

1. **Cantidad de neuronas de la capa de entrada**: **7 neuronas**
   - Una por cada característica del dataset (Area, Perimetro, Compacidad, LongNucleo, AnchoNucleo, Asimetria, LongSurco)

2. **Cantidad de neuronas de la capa de salida**: **3 neuronas**
   - Una por cada variedad de trigo a clasificar (Kama, Rosa, Canadiense)

3. **Cantidad de pesos (arcos) con una capa oculta de 4 neuronas**: **47 pesos**
   - 32 pesos entre entrada y capa oculta (7×4 + 4 bias)
   - 15 pesos entre capa oculta y salida (4×3 + 3 bias)
   - Total: 32 + 15 = 47 pesos